In [ ]:
import torch
import os
import sys
import matplotlib.pyplot as plt
import argparse
import numpy as np
from tqdm.auto import tqdm
import torch.nn.functional as F

sys.path.append("../../")
from src.filepath import ABSOLUTE_PATH
from src.model.diffusion import GaussianDiffusion
from src.model.UNet2d import Unet2D
from src.model.fno import FNO2D
from src.inference.compose import compose_diffusion
from src.utils.utils import plot_compare_2d, relative_error
from src.train.reaction_diffusion import cond_emb, renormalize
from src.train.reaction_diffusion import normalize_to_neg_one_to_one as normalize

In [2]:
model_type = "FNO"

## load model

In [3]:
dim = 24
out_dim = 1
channel = 3
nx = 20
diffusion_step = 250
device = "cuda"
if model_type == "Unet":
    model1 = Unet2D(dim=dim, cond_emb=cond_emb(), out_dim=out_dim, dim_mults=(1, 2), channels=channel)
    model2 = Unet2D(dim=dim, cond_emb=cond_emb(), out_dim=out_dim, dim_mults=(1, 2), channels=channel)
elif model_type == "ViT":
    model1 = ViT(
        seq_len=20,
        patch_size=2,
        dim=64,
        depth=2,
        heads=8,
        mlp_dim=128,
        cond_emb=cond_emb(),
        Time_Input=True,
        dropout=0.0,
        emb_dropout=0.0,
        channels=20,
        out_channels=9,
    ).to("cuda")
    model2 = ViT(
        seq_len=20,
        patch_size=2,
        dim=64,
        depth=2,
        heads=8,
        mlp_dim=128,
        cond_emb=cond_emb(),
        Time_Input=True,
        dropout=0.0,
        emb_dropout=0.0,
        channels=20,
        out_channels=9,
    ).to("cuda")
elif model_type == "FNO":
    model1 = FNO2D(
        in_channels=channel,
        out_channels=out_dim,
        nr_fno_layers=4,
        fno_layer_size=24,
        fno_modes=[6, 12],
        time_input=True,
        cond_emb=cond_emb(),
    )
    model2 = FNO2D(
        in_channels=channel,
        out_channels=out_dim,
        nr_fno_layers=4,
        fno_layer_size=24,
        fno_modes=[6, 12],
        time_input=True,
        cond_emb=cond_emb(),
    )
diffusion1 = GaussianDiffusion(model1, seq_length=(out_dim, 10, nx), timesteps=diffusion_step, auto_normalize=False).to(
    device
)
diffusion2 = GaussianDiffusion(model2, seq_length=(out_dim, 10, nx), timesteps=diffusion_step, auto_normalize=False).to(
    device
)

In [ ]:
if model_type == "Unet":
    diffusion1.load_state_dict(torch.load("../../results/reaction_diffusion/diffusionUnetu10000/model.pt")["model"])
    diffusion2.load_state_dict(torch.load("../../results/reaction_diffusion/diffusionUnetv10000/model.pt")["model"])
elif model_type == "ViT":
    diffusion1.load_state_dict(torch.load("../../results/reaction_diffusion/diffusionViTu10000/model.pt")["model"])
    diffusion2.load_state_dict(torch.load("../../results/reaction_diffusion/diffusionViTv10000/model.pt")["model"])
elif model_type == "FNO":
    diffusion1.load_state_dict(torch.load("../../results/reaction_diffusion/diffusionFNOu10000/model.pt")["model"])
    diffusion2.load_state_dict(torch.load("../../results/reaction_diffusion/diffusionFNOv10000/model.pt")["model"])

validation of u

In [ ]:
# for i in range(10, 101):
#     diffusion2.load_state_dict(
#         torch.load("../../results/reaction_diffusion/diffusionFNOu10000/model-" + str(i) + ".pt")["model"]
#     )
data = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_u_from_v_u.npy")).float().to("cuda")[9000:]
)
cond = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_u_from_v_v.npy")).float().to("cuda")[9000:]
)

data = torch.tensor(data).unsqueeze(1)

cond1 = torch.tensor(cond).unsqueeze(1)

cond2 = data[:, :, 0:1].clone().expand(-1, -1, data.shape[2], -1)

cond = torch.concat((cond1, cond2), dim=1)

u_pred = renormalize(diffusion1.sample(cond.shape[0], cond=[normalize(cond)]))

rmse = relative_error(data, u_pred)

In [ ]:
random_n = np.random.randint(0, data.shape[0])
plot_compare_2d(
    true_d=data[random_n, 0],
    pred_d=u_pred[random_n, 0],
)
# plot_compare_2d(data, un(u_pred))

validation of v

In [ ]:
# for i in range(51, 91):
#     diffusion2.load_state_dict(
#         torch.load("../../results/reaction_diffusion/diffusionFNOv10000/model-" + str(i) + ".pt")["model"]
#     )
data = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_v_from_u_v.npy")).float().to("cuda")[9000:]
)
cond = (
    torch.tensor(np.load("../../data/reaction_diffusion/reaction_diffusion_v_from_u_u.npy")).float().to("cuda")[9000:]
)


data = torch.tensor(data).unsqueeze(1)


cond1 = torch.tensor(cond).unsqueeze(1)


cond2 = data[:, :, 0:1].clone().expand(-1, -1, data.shape[2], -1)


cond = torch.concat((cond1, cond2), dim=1)


v_pred = renormalize(diffusion2.sample(cond.shape[0], cond=[normalize(cond)]))


rmse = relative_error(data, v_pred)
print(rmse)

In [ ]:
random_n = np.random.randint(0, data.shape[0])
plot_compare_2d(
    true_d=data[random_n, 0],
    pred_d=(v_pred)[random_n, 0],
)
# plot_compare_2d(data, un(u_pred))

## compose

In [ ]:
data = (
    torch.tensor(np.load(ABSOLUTE_PATH + "/data/reaction_diffusion/reaction_diffusion_uv.npy"))
    .float()
    .to("cuda")[:2000]
)
# data = (data + 5) / 10

data = data.permute(0, 2, 1)

# data1 = np.load('../../data/reaction_diffusion_u_from_v_u.npy')

u, v = data[..., :20].unsqueeze(1), data[..., 20:].unsqueeze(1)

u_intial, v_intial = u[:, :, 0:1].expand(-1, -1, 10, -1), v[:, :, 0:1].expand(-1, -1, 10, -1)

u.shape, u_intial.shape

In [15]:
def update_u(
    alpha, t, model, field_noise, mult_p_estimate, mult_p_estimate_before, other_condition, normalize, renormalize
):
    weight_field = []
    for i in range(len(mult_p_estimate)):
        weight_field.append(alpha * mult_p_estimate[i] + (1 - alpha) * mult_p_estimate_before[i])
    intial_u, intial_v = other_condition[0], other_condition[1]
    cond = [torch.concat((weight_field[1], intial_u), dim=1)]
    field_noise_next, x0 = model.p_sample(field_noise, t, cond)
    return field_noise_next, x0


def update_v(
    alpha, t, model, field_noise, mult_p_estimate, mult_p_estimate_before, other_condition, normalize, renormalize
):
    weight_field = []
    for i in range(len(mult_p_estimate)):
        weight_field.append(alpha * mult_p_estimate[i] + (1 - alpha) * mult_p_estimate_before[i])
    intial_u, intial_v = other_condition[0], other_condition[1]
    cond = [torch.concat((weight_field[0], intial_v), dim=1)]
    field_noise_next, x0 = model.p_sample(field_noise, t, cond)
    return field_noise_next, x0

In [ ]:
model_lis = [diffusion1, diffusion2]
mult_p = compose_diffusion(
    model_list=model_lis,
    shape=[(u.shape[0], 1, 10, 20), (u.shape[0], 1, 10, 20)],
    other_condition=[normalize(u_intial), normalize(v_intial)],
    update_f=[update_u, update_v],
    normalize_f=[normalize, normalize],
    unnormalize_f=[renormalize, renormalize],
    num_iter=2,
    device="cuda",
)

In [ ]:
mult_p_true = torch.concat((u, v), dim=1)
mult_p_pred = renormalize(torch.concat((mult_p[0], mult_p[1]), dim=1))


mult_p_true.shape, mult_p_pred.shape

In [ ]:
relative_error(mult_p_pred[:, 0], mult_p_true[:, 0]), relative_error(mult_p_pred[:, 1], mult_p_true[:, 1])

iter 1  
use trick 1.70e-2 1.95e-2   
unuse trick 1.69e-2 1.95e-2

iter 2  
use trick 1.70e-2 1.94e-2   
unuse trick 1.68e-2 1.94e-2

(tensor(8.9275e-06, device='cuda:0'),
 tensor(1.4842e-05, device='cuda:0'),
 tensor(3.0127e-06, device='cuda:0'))

In [ ]:
random_n = np.random.randint(0, data.shape[0])
random_n = -1


plot_compare_2d(
    true_d=mult_p_true[random_n, 1],
    pred_d=mult_p_pred[random_n, 1],
    savep=ABSOLUTE_PATH + "/results/reaction_diffusion/diffusion.pdf",
)